In [4]:
import gzip
import pandas as pd

In [5]:
soft_file = "/home/mlopez/Desktop/alzheimer/data/GSE33000/GSE33000_family.soft.gz"

In [7]:
sample_metadata = []
current_sample = None
characteristics = {}

with gzip.open(soft_file, 'rt', encoding='utf-8') as f:
    for line in f:
        line = line.strip()

        if line.startswith("^SAMPLE"):
            if current_sample:
                sample_metadata.append(characteristics)
            current_sample = line.split(" = ")[1]
            characteristics = {"Accession": current_sample}

        elif line.startswith("!Sample_title"):
            characteristics["Title"] = line.split(" = ")[1]

        elif line.startswith("!Sample_source_name_ch1"):
            characteristics["Source"] = line.split(" = ")[1]

        elif line.startswith("!Sample_organism_ch1"):
            characteristics["Organism"] = line.split(" = ")[1]

        elif line.startswith("!Sample_characteristics_ch1") or line.startswith("!Sample_characteristics_ch2"):
            text = line.split(" = ")[1]
            try:
                key, value = text.split(": ")
                characteristics[key.strip()] = value.strip()
            except ValueError:
                characteristics.setdefault("Other_Characteristics", []).append(text)

# Añadir el último sample
if current_sample:
    sample_metadata.append(characteristics)

# Crear el DataFrame
df_sample_metadata = pd.DataFrame(sample_metadata)

# Mostrar columnas útiles
cols_to_check = ["Accession", "Title", "Source", "Organism"]
cols_to_check += [col for col in df_sample_metadata.columns if any(k in col.lower() for k in ["tissue", "region", "disease", "status"])]

# Visualizar columnas seleccionadas
df_sample_metadata[cols_to_check].head()

,Accession,Title,Source,Organism,tissue,disease status
0,GSM1423780,PFC_1,HBTRC_PF_Pool_1,Homo sapiens,prefrontal cortex brain,Alzheimer's disease
1,GSM1423781,PFC_2,HBTRC_PF_Pool_2,Homo sapiens,prefrontal cortex brain,Alzheimer's disease
2,GSM1423782,PFC_3,HBTRC_PF_Pool_3,Homo sapiens,prefrontal cortex brain,Alzheimer's disease
3,GSM1423783,PFC_4,HBTRC_PF_Pool_4,Homo sapiens,prefrontal cortex brain,Alzheimer's disease
4,GSM1423784,PFC_5,HBTRC_PF_Pool_5,Homo sapiens,prefrontal cortex brain,Alzheimer's disease


In [ ]:
df_sample_metadata.to_csv("/home/mlopez/Desktop/alzheimer/results//GSE28146_metadata.csv", index=False)